In [1]:
import torch, dgl
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from dgl.data import DGLDataset, CoraGraphDataset
import dgl.function as fn
import re
import numpy as np
import pandas as pd
import os.path as osp
from colorama import Fore
from glob import glob


In [2]:
class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    
    def __init__(self, in_feat:int, out_feat:int):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata['h'] = h
            # update_all is a message passing API.
            g.update_all(message_func = fn.copy_u('h', 'm'), 
                         reduce_func = fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)


class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [ ]:
class DkkdGraphDataset(DGLDataset):
    def __init__(self, root:str='dataset/graph_DKKD'):
        super().__init__(name='dataset/Karate_Club')
        self.root = root
        self.edges = glob(osp.join(root, '*.edges.csv'))
        self.nodes_feat = glob(osp.join(root, '*.nfeat.npy'))
        self.nodes_label = glob(osp.join(root, '*.idx.csv'))

    def __len__(self): return len(self.edges)
    def __getitem__(self, i):
        edgep = self.edges[i]
        nodes_feat = np.load(re.sub('.edges.csv$', '.nfeat.npy', edgep))
        nodes_label = pd.read_csv(
            re.sub('.edges.csv$', '.nfeat.npy', edgep), encoding='utf-8')
        
        def _get_n_nodes():
            r"""
            tính và kiểm tra số thứ tự của node
            """
            n_nodes = nodes_label['Id'].to_list()
            for i, idx in enumerate(n_nodes):
                assert i == idx, 'i != idx'
            return len(n_nodes)
        n_nodes = _get_n_nodes()
        
        nodes_label = nodes_label['label'].astype('category').cat.codes.to_list()
        edge = pd.read_csv(edgep, encoding='utf-8')
        g = dgl.graph((edge['src'], edge['dst']), num_nodes=n_nodes)
        g = dgl.to_bidirected(g)
        g.ndata['feat'] = torch.from_numpy(nodes_feat)
        g.ndata['label'] = torch.tensor(nodes_label)
        g.ndata['train_mask'] = torch.ones(n_nodes, dtype=torch.bool)
        g.ndata['val_mask'] = torch.zeros(n_nodes, dtype=torch.bool)
        g.ndata['test_mask'] = torch.zeros(n_nodes, dtype=torch.bool)
        
        return self.graph
    
    def process(self):
        #NOTE download data
        # urlretrieve('https://data.dgl.ai/tutorial/dataset/members.csv', './members.csv')
        # urlretrieve('https://data.dgl.ai/tutorial/dataset/interactions.csv', './interactions.csv')
        nodes_data = pd.read_csv('dataset/Karate_Club/members.csv')
        edges_data = pd.read_csv('dataset/Karate_Club/interactions.csv')
        n_nodes  = nodes_data.shape[0]
        
        node_labels  = torch.tensor(nodes_data['Club'].astype('category').cat.codes.to_list())
        node_features = torch.from_numpy(nodes_data['Age'].to_numpy()).reshape(n_nodes, 1)
        edge_features = torch.from_numpy(edges_data['Weight'].to_numpy())
        edges_src = torch.from_numpy(edges_data['Src'].to_numpy())
        edges_dst = torch.from_numpy(edges_data['Dst'].to_numpy())
        
        #NOTE create graph with nodes and edges feature
        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=n_nodes)
        self.graph = dgl.to_bidirected(self.graph)      # convert to undirected
        self.graph.ndata['feat'] = node_features        #NOTE learnable
        self.graph.ndata['label'] = node_labels
        self.graph.ndata['node_features'] = nn.Parameter(torch.randn(self.graph.num_nodes(), 10))

        self.graph.edata['weight'] = edge_features

        #NOTE If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.

        n_train = int(n_nodes * 0.6)
        n_val  = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask  = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask  = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True
        self.graph.ndata['train_mask'] = train_mask
        self.graph.ndata['val_mask'] = val_mask
        self.graph.ndata['test_mask'] = test_mask